In [3]:
import json
import pprint as pp
import praw
from datetime import datetime
import os
from dotenv import load_dotenv

# --- 1. 환경 변수 로드 ---
# .env 파일에서 자격 증명을 로드합니다.
try:
    env_path = os.path.join('.env')
    load_dotenv(dotenv_path=env_path)

    client_id = os.getenv('REDDIT_CLIENT_ID')
    client_secret = os.getenv('REDDIT_CLIENT_SECRET')

    if not client_id or not client_secret:
        print("REDDIT_CLIENT_ID 또는 REDDIT_CLIENT_SECRET이 .env 파일에 없습니다.")
        exit()
    
    print(f"Client ID: {client_id[:5]}...") # Client ID 로드 확인

except Exception as e:
    print(f".env 파일 로드 중 오류 발생: {e}")
    exit()

# --- 2. PRAW 인증 (단순화) ---
# PRAW는 ID와 Secret만으로 인증을 자동으로 처리합니다.
# requests를 사용한 수동 토큰 발급은 전혀 필요하지 않습니다.
try:
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent="ydg06081-top-search/1.0" # User-Agent는 필수
    )
except Exception as e:
    print(f"PRAW 인증 실패: {e}")
    exit()

# --- 3. 날짜 설정 (필터링용) ---
# 우리가 원하는 실제 기간
start_date_str = "2024-10-01"
end_date_str = "2025-10-01"
start_ts = int(datetime.strptime(start_date_str, "%Y-%m-%d").timestamp())
end_ts = int(datetime.strptime(end_date_str, "%Y-%m-%d").timestamp())

# --- 4. 데이터 수집 (.top() 전략 사용) ---

#==================================서브레딧===========================================
subreddit_name = "stocks" # 사용자님이 마지막으로 사용하신 subreddit
#=====================================================================================

subreddit = reddit.subreddit(subreddit_name)

print(f"--- r/{subreddit_name}에서 [{start_date_str} ~ {end_date_str}] 기간의 [인기 글] 수집 시작 ---")

collected_posts_dict = {} # 중복 제거를 위한 딕셔너리 (post.id -> post 객체)

# [전략 1: "지난 1년" (Year) 인기 글 수집]
print("[전략 1] '지난 1년' 인기 글(Top 1000)을 확인합니다...")
try:
    # time_filter="year"는 오늘(2025-11-18) 기준 지난 1년치를 가져옴
    for post in subreddit.top(time_filter="year", limit=1000):
        post_ts = post.created_utc
        # 가져온 인기 글을 우리가 원하는 날짜 범위(2024-10 ~ 2025-10)로 다시 필터링
        if start_ts <= post_ts <= end_ts:
            if post.id not in collected_posts_dict:
                collected_posts_dict[post.id] = post
except Exception as e:
    print(f"수집 중 오류 발생 (Year): {e}")

# [전략 2: "역대" (All-Time) 인기 글 수집]
print("[전략 2] '역대' 인기 글(Top 1000)을 확인합니다...")
try:
    # time_filter="all"는 역대 인기글 1000개를 가져옴
    for post in subreddit.top(time_filter="all", limit=1000):
        post_ts = post.created_utc
        # 가져온 인기 글을 우리가 원하는 날짜 범위(2024-10 ~ 2025-10)로 다시 필터링
        if start_ts <= post_ts <= end_ts:
            if post.id not in collected_posts_dict:
                collected_posts_dict[post.id] = post
except Exception as e:
    print(f"수집 중 오류 발생 (All): {e}")

# --- 5. 최종 리스트 변환 (사용자님 형식 유지) ---
posts = []
print(f"\n--- 총 {len(collected_posts_dict)}개의 고유한 인기 글 수집 완료 ---")
print("수집된 데이터를 리스트 형식으로 변환합니다...")

# 딕셔너리에 모은 post 객체들을 리스트로 변환
for post in collected_posts_dict.values():
    post_time = datetime.fromtimestamp(post.created_utc)
    posts.append({
        "title": post.title,
        "url": post.url,
        "created": post_time,
        "score": post.score,
        "num_comments": post.num_comments,
        "author": str(post.author),
        "selftext": post.selftext
    })

# 날짜순으로 정렬 (오래된 순)
posts.sort(key=lambda x: x['created'])

# --- 6. 결과 요약 출력 ---
if posts:
    print(f"\n✅ Reddit 크롤링 완료. 총 {len(posts)}개 수집.")
    print(f"  [가장 오래된 글] {posts[0]['created'].strftime('%Y-%m-%d')}: {posts[0]['title'][:50]}...")
    print(f"  [가장 최신 글]   {posts[-1]['created'].strftime('%Y-%m-%d')}: {posts[-1]['title'][:50]}...")
else:
    print("✅ Reddit 크롤링 완료. 해당 기간에 수집된 인기 글이 없습니다.")

# --- 7. JSON 파일로 저장 ---
if posts:
    output_filename = f"{subreddit_name}_{start_date_str}_to_{end_date_str}.json"
    print(f"\n💾 {output_filename} 파일로 저장 중...")
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            # datetime 객체는 JSON으로 바로 저장할 수 없으므로
            # default=str을 사용하여 문자열로 변환합니다.
            json.dump(posts, f, ensure_ascii=False, indent=4, default=str)
        print(f"✅ {output_filename} 파일 저장 완료!")
    except Exception as e:
        print(f"❌ 파일 저장 중 오류 발생: {e}")

Client ID: P3LdN...
--- r/stocks에서 [2024-10-01 ~ 2025-10-01] 기간의 [인기 글] 수집 시작 ---
[전략 1] '지난 1년' 인기 글(Top 1000)을 확인합니다...
[전략 2] '역대' 인기 글(Top 1000)을 확인합니다...

--- 총 232개의 고유한 인기 글 수집 완료 ---
수집된 데이터를 리스트 형식으로 변환합니다...

✅ Reddit 크롤링 완료. 총 232개 수집.
  [가장 오래된 글] 2024-11-12: Elon Musk’s Trump bet has paid off so well that Te...
  [가장 최신 글]   2025-09-29: How will Microsoft stock react to this news on Mon...

💾 stocks_2024-10-01_to_2025-10-01.json 파일로 저장 중...
✅ stocks_2024-10-01_to_2025-10-01.json 파일 저장 완료!
